In [69]:
import pandas as pd
import numpy as np

In [70]:
df = pd.read_csv('train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [71]:
df.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [72]:
df['keyword'] = df['keyword'].fillna(df['keyword'].mode()[0])
df['location'] = df['location'].fillna(df['location'].mode()[0])
df.drop('id', axis=1, inplace=True)
df.isna().sum()

keyword     0
location    0
text        0
target      0
dtype: int64

In [73]:
df.drop('location', axis=1, inplace=True)
df.drop('keyword', axis=1, inplace=True)

In [74]:
from sentence_transformers import SentenceTransformer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

transformer = SentenceTransformer("all-MiniLM-L6-v2", device=device)

embeddings = transformer.encode(df['text'], device=device)

df['embeddings'] = [embedding for embedding in embeddings]

df.drop('text', axis=1, inplace=True)

Using device: cuda


In [75]:
df.head()

,target,embeddings
0,1,"[-0.014600068, 0.046868287, 0.11572448, 0.0185..."
1,1,"[0.040229227, 0.03801415, -0.0064313635, 0.024..."
2,1,"[0.1313757, 0.012401276, 0.06718614, 0.0854625..."
3,1,"[0.094014056, -0.0375565, 0.029119445, 0.02958..."
4,1,"[-0.04094861, 0.08828365, 0.07735549, 0.086615..."


In [77]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

# Convert embeddings to PyTorch tensors and move to GPU
X = torch.tensor(embeddings, dtype=torch.float32).to(device)
y = torch.tensor(df['target'].values, dtype=torch.long).to(device)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

class FCNN(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        x = torch.tanh_(self.fc1(x))
        x = torch.tanh_(self.fc2(x))
        x = self.fc3(x)
        return x

model = FCNN(embeddings.shape[1], 128, 64, 2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training and evaluation code can be similar to the BiLSTM example.
num_epochs = 20

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

# Evaluation
model.eval()
with torch.no_grad():
    test_output = model(X_test)
    _, predicted = torch.max(test_output, 1)
    accuracy = (predicted == y_test).float().mean()
    print(f"Test Accuracy: {accuracy.item() * 100:.2f}%")

Epoch 1/20, Loss: 0.6975936889648438
Epoch 2/20, Loss: 0.6909628510475159
Epoch 3/20, Loss: 0.6846793293952942
Epoch 4/20, Loss: 0.6784937977790833
Epoch 5/20, Loss: 0.6721362471580505
Epoch 6/20, Loss: 0.6653664708137512
Epoch 7/20, Loss: 0.6580127477645874
Epoch 8/20, Loss: 0.6499589681625366
Epoch 9/20, Loss: 0.6411325931549072
Epoch 10/20, Loss: 0.6314940452575684
Epoch 11/20, Loss: 0.6210269331932068
Epoch 12/20, Loss: 0.6097334623336792
Epoch 13/20, Loss: 0.5976397395133972
Epoch 14/20, Loss: 0.5848064422607422
Epoch 15/20, Loss: 0.5713421106338501
Epoch 16/20, Loss: 0.557418704032898
Epoch 17/20, Loss: 0.5432823896408081
Epoch 18/20, Loss: 0.5292520523071289
Epoch 19/20, Loss: 0.5156936049461365
Epoch 20/20, Loss: 0.5029703974723816
Test Accuracy: 79.32%


In [78]:
# Print the f1 score
from sklearn.metrics import f1_score

y_pred = predicted.cpu().numpy()
y_test = y_test_tensor.cpu().numpy()

print(f1_score(y_test, y_pred))

0.756003098373354


In [79]:
print(f1_score(y_test, y_pred))

0.756003098373354


In [80]:
test_df = pd.read_csv('test.csv')

test_df.drop('location', axis=1, inplace=True)
test_df.drop('keyword', axis=1, inplace=True)

embeddings = transformer.encode(test_df['text'])

test_df['embeddings'] = [embedding for embedding in embeddings]

test_df.drop('text', axis=1, inplace=True)

# Convert embeddings to array without reshaping to 3D
X_test = np.array(test_df['embeddings'].tolist())

# Set the model to evaluation mode
model.eval()
with torch.no_grad():
    # Pass X_test as a 2D tensor directly
    test_output = model(torch.tensor(X_test, dtype=torch.float32).to(device))
    _, predicted = torch.max(test_output, 1)

y_pred = predicted.cpu().numpy()

# Save predictions
test_df['target'] = y_pred
test_df.drop('embeddings', axis=1, inplace=True)
test_df.to_csv('submission.csv', index=False)